<a href="https://colab.research.google.com/github/mahekp05/Reinforcement-Learning-Projects/blob/main/Mario.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Install Packages and Dependencies

*   **gymnaisum**: toolkit for developing and comparing reinforcement learning algorithms. It provides various environments that simulate real-world scenarios, which are essential for testing and training RL models.
*   **pygame**: writing video games; in reinforcement learning it is used to create custom environments
*   **numpy**: library for numerical computation in python and provides support for arrays, matrices, and many mathematical functions essential for implementing and testing algorithms
*   **imageio and imageio_ffmpeg**: libararies used to read and write images and videos -- useful to render environment and create visualization of our reinforcement learning model and training episodes on collabnotes as live redering is not possible
*   **pyglet**: A cross-platform windowing and multimedia library -- used for rendering graphics in reinforcement learning environments
*   **tqdm**: creates progress bars to help monitor progress of training and lenghty episodes

In [ ]:
!pip install ale-py
!pip install gymnasium[atari]
!pip install gymnasium[accept-rom-license]
!pip install pygame
!pip install numpy
!pip install pickle5
!pip install imageio
!pip install imageio_ffmpeg
!pip install pyglet==1.5.1
!pip install tqdm

* stable_baselines3:

In [ ]:
!pip install stable_baselines3


* **tensorflow**: A popular library for building and training machine learning models. tf.keras is TensorFlow's high-level API for defining and training neural networks
* **sudo apt-get update**: Updates the package lists for the repositories, ensuring you get the latest information on the newest versions of packages and their dependencies.
* **sudo apt-get install -y python3-opengl**: Installs the Python bindings for OpenGL, which are necessary for running graphical applications that use OpenGL.
* **sudo apt-get install -y ffmpeg xvfb**: Installs FFmpeg, a complete solution for recording, converting, and streaming audio and video, and Xvfb, a display server that performs graphical operations in memory without showing any screen output.
* **pip3 install pyvirtualdisplay**: Installs the pyvirtualdisplay package using pip, which allows you to create and manage virtual displays.

In [ ]:
import tensorflow as tf

In [ ]:
!sudo apt-get update
!sudo apt-get install -y python3-opengl
!apt install ffmpeg xvfb
!pip3 install pyvirtualdisplay

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading pack

In [ ]:
import os
os.kill(os.getpid(),9)

##2. Import Dependencies and Packages

In [ ]:
import numpy as np
import gymnasium as gym
import random
import os
import imageio
import tqdm

import pickle5 as pickle
from tqdm.notebook import tqdm

In [ ]:
from stable_baselines3 import DQN

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


* **Sequential**: A type of model in Keras where layers are stacked sequentially.
* **Dense**: A fully connected layer where each neuron is connected to every neuron in the previous layer.
* **Conv2D**: A 2D convolutional layer, which is commonly used for processing image data.
* **Flatten**: A layer that flattens the input, converting it from a multi-dimensional tensor to a 1D tensor.
* **Adam**: An optimization algorithm used for training the model.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten
from tensorflow.keras.optimizers import Adam

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


##3. Virtual Display

In [ ]:
from pyvirtualdisplay import Display
display = Display(visible=0,size=(1400,900))
display.start()

##4. Create Environment

*Initialize Environment*

In [ ]:
env = gym.make("ALE/MarioBros-v5", render_mode = "rgb_array")

*Environment State Space*


In [ ]:
state_space = env.observation_space
print("State Space:", state_space)


State Space: Box(0, 255, (210, 160, 3), uint8)


(0,255) -> (min, max) number of elements observation array can take

(210,160,3) -> (height of image [pix], width of image [pix], color channels #)

(uint8) -> unsigned 8-bit int

*Environment Action Space*

In [ ]:
action_space = env.action_space.n
print("Action Space:", action_space)

Action Space: 18


18 possible action spaces

#####Initialize Q-table
* represents the expected future rewards for state-action pairs and requires discrete states and actions.

NOT SIMPLE FOR BOX (CONTINOUS ENVIRONMENTS) AS USUALLY FOR DISCRETE SPACES

INSTEAD USE function approximation methods like Q-learning with nueral  networks to estimate q-values
1. Deep Q-learning (DQN) -- best for continious state spaces and discrete action spaces
2. Policy Graident Methods
3. Actor-Critic Methods
4. Deep Deterministic Policy Gradient




Deep Q-learning (DQN) Key Components
1. Q-network - approximates the Q-value
2. Experience Relay - buffer that stores past experiences (state, action, reward, next state)
      * Training on random samples from this buffer helps in stabilizing the learning process by breaking the correlation between consecutive experiences.

3. Target Network - updated less frequently. It helps to stabilize the training process by providing consistent target Q-values for the Bellman update
4. Epsilon Greedy Strategy - A policy for balancing exploration and exploitation

How DQN Handles Continuous State Spaces
* Convolutional Neural Networks (CNNs): to process the image data
    * CNNs are effective at extracting features from image data and learning spatial hierarchies.
* State Representation: to produce feature maps
    * These feature maps are then used to predict Q-values for discrete actions.


In [ ]:
def initialize_dq_nueral_network(state_shape, action_num):
  """
    Initialize a Deep Q-Network (DQN) neural network for reinforcement learning.

    Parameters:
    - state_shape: Tuple representing the shape of the input state (e.g., image dimensions).
    - action_num: Integer representing the number of possible actions in the environment.

    Returns:
    - model: A compiled TensorFlow Keras Sequential model.
    """

  model = Sequential()

  #Add convolution layers for image data
  #(input dimensions, kernel size, strides,activation function, state_shape)

  # Add a convolutional layer to process image data
  model.add(Conv2D(32,(8,8), strides = (4,4), activation = "relu", input_shape=state_shape))
  # Add a second convolutional layer
  model.add(Conv2D(64,(4,4),strides = (2,2), activation = "relu"))
  # Add a third convolutional layer
  model.add(Conv2D(64,(3,3), activation = "relu"))

  # Flatten the output from the convolutional layers --- convert the 3D tensor into a 1D tensor for the fully connected
  model.add(Flatten())

  #Add a fully connected (dense) layer
  model.add(Dense(512, activation = "relu"))

  #output layers: one q-value for each action
  #(total possible actions = # of nuerons,  linear activation to ouput q-value for each action)
  model.add(Dense(action_num, activation = "linear"))

  #compile model -- Adam optimizer with a learning rate of 0.001
  #Mean Squared Error (MSE) loss function for training the model
  model.compile(optimizer = Adam(learning_rate=0.001), loss = "mse")

  return model

model.add(Conv2D(32,(8,8), strides = (4,4), activation = "relu", state_space=state_space))
* 32: Number of filters (or kernels) in this convolutional layer.
* (8, 8): Size of the convolutional kernels.
* strides=(4, 4): The stride of the convolution --  determines how much the filter moves across the input image.
* activation='relu': Activation function used to introduce non-linearity.
    * ReLU (Rectified Linear Unit) helps the network learn complex patterns.
* input_shape=input_shape: Specifies the shape of the input data

----
* Input Dimensions: Directly from your data.
* Kernel Size: Selected based on design choices and experimentation.
* Strides: Chosen based on how aggressively you want to reduce spatial dimensions.
* Padding: Affects whether the output size is reduced or preserved.


* Flatten Layer: Flattens the 3D output of the convolutional layers into a 1D vector.
* Dense Layer: Fully connected layer with 512 neurons and ReLU activation.
* Output Layer: Dense layer with a number of neurons equal to the number of possible actions, with linear activation.
* Compile: Compiles the model using the Adam optimizer and mean squared error loss function.


In [ ]:
state_shape = env.observation_space.shape
action_num = env.action_space.n

Qnetwork_mario = initialize_dq_nueral_network(state_shape=state_shape, action_num=action_num)
print(Qnetwork_mario.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 51, 39, 32)        6176      
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 18, 64)        32832     
                                                                 
 conv2d_2 (Conv2D)           (None, 22, 16, 64)        36928     
                                                                 
 flatten (Flatten)           (None, 22528)             0         
                                                                 
 dense (Dense)               (None, 512)               11534848  
                                                                 
 dense_1 (Dense)             (None, 18)                9234      
                                                                 
Total params: 11620018 (44.33 MB)
Trainable params: 1162

##5.Training

####1. create replay buffer
* deque DS used to store and manage agent's past experiences during interaction with environment

Each experience stored as tuple in form of (state, actio, reward, next_stae, done)

*Randomization helps break correlation between consecutive episodes, stabilizing training process

*Atend can learn from past experience where experiences are costly or slow to accumulate

In [ ]:
from collections import deque

class ReplayBuffer:
  def __init__(self, max_size):
    #older experiences automatically removed when buffer reaches size
    self.buffer = deque(maxlen = max_size)

  #add new experience to buffer
  def add(self, experience):
    #if full, oldest experience discarded
    self.buffer.append(experience)

  #sample batch of experiences from buffer
  def sample(self, batch_size):
    #randomly samples batch which is used to train nueral network [note: np = numpy]
    indices = np.random.choice(len(self.buffer),batch_size, replace =False)
    batch = [self.buffer[idx] for idx in indices]

    states, actions, rewards, next_states, dones = map(np.array, zip(*batch))
    return states, actions, rewards, next_states, dones

  #return number of experiences in buffer
  def __len__(self):
    return len(self.buffer)


####2. Training with Experience Relay



In [ ]:
def epsilon_greedy_policy(Qnetwork, state, epsilon, action_space):
  rand_num = np.random.rand()

  if rand_num <= epsilon:
    action = np.random.randint(0,action_space) #Explore
   # print(f"Exploration: action chosen randomly: {action}")
  else: #Exploit
    state_input = np.expand_dims(state, axis=0)
    #print(f"State shape before prediction: {state_input.shape}")
    Qvalues = Qnetwork.predict(state_input)
    #print(f"Q-values predicted: {Qvalues}")
    action = np.argmax(Qvalues[0])
    #print(f"Exploitation: action chosen by Q-network: {action}")

  return action


Hyper-parameters

In [ ]:
#training parameters
n_training_episodes = 10
learning_rate = 0.7

#Evaluation parameters
n_eval_episodes = 100

#Environment parameters
env_id = "ALE/MarioBros-v5"
gamma = 0.99

#Exploration parameters
epsilon = 1.0
epsilon_decay = 0.995
min_epsilon = 0.01
target_update_freq = 10
batch_size = 64

In [ ]:
def train_dqn(env, q_network, target_network, replay_buffer, episodes, batch_size, gamma,
              epsilon, epsilon_decay, min_epsilon, target_update_freq):
    for episode in tqdm(range(episodes)):
        state = env.reset()
        if isinstance(state, tuple):
            state = state[0]
        state = np.array(state)  # Ensure state is a numpy array
        done = False
        total_reward = 0

        while not done:
            action = epsilon_greedy_policy(q_network, state, epsilon, action_space)
            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated
            if isinstance(next_state, tuple):
                next_state = next_state[0]
            next_state = np.array(next_state)  # Ensure next_state is a numpy array
            replay_buffer.add((state, action, reward, next_state, done))
            state = next_state
            total_reward += reward

            if len(replay_buffer) > batch_size:
                states, actions, rewards, next_states, dones = replay_buffer.sample(batch_size)
                targets = q_network.predict(states)
                target_vals = target_network.predict(next_states)
                for i in range(batch_size):
                    if dones[i]:
                        targets[i][actions[i]] = rewards[i]
                    else:
                        targets[i][actions[i]] = rewards[i] + gamma * np.amax(target_vals[i])
                q_network.fit(states, targets, epochs=1, verbose=0)

            if epsilon > min_epsilon:
                epsilon *= epsilon_decay

        if episode % target_update_freq == 0:
            target_network.set_weights(q_network.get_weights())

        print(f"Episode: {episode}/{episodes}, Total reward: {total_reward}, Epsilon: {epsilon:.2f}")


####3. Train

##6. Model

In [ ]:
#Initialize environment
env1 = gym.make(env_id, render_mode = "rgb_array")
state_shape = env.observation_space.shape
action_num = env.action_space.n

#Initialize Q-netwrok and target netwrok
Qnetwork_mario = initialize_dq_nueral_network(state_shape, action_num)
target_network = initialize_dq_nueral_network(state_shape, action_num)
target_network.set_weights(Qnetwork_mario.get_weights())

#Initialize replay buffer
replay_buffer = ReplayBuffer(1000)

train_dqn(env1, Qnetwork_mario, target_network, replay_buffer, n_training_episodes, batch_size, gamma,
          epsilon, epsilon_decay, min_epsilon, target_update_freq)



  0%|          | 0/10 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
2/2 [==============================] - 0s 83ms/step
Episode: 0/10, Total reward: 0.0, Epsilon: 0.02
2/2 [==============================] - 0s 94ms/step
Episode: 1/10, Total reward: 0.0, Epsilon: 0.01
2/2 [==============================] - 0s 98ms/step


In [ ]:
# Define the model dictionary
model = {
    "env_id": env_id,
    "state_shape": state_shape,
    "action_num": action_num,
    "Qnetwork_mario": Qnetwork_mario,
    "target_network": target_network,
    "replay_buffer": replay_buffer,
    "n_training_episodes": n_training_episodes,
    "learning_rate": learning_rate,
    "n_eval_episodes": n_eval_episodes,
    "gamma": gamma,
    "epsilon": epsilon,
    "epsilon_decay": epsilon_decay,
    "min_epsilon": min_epsilon,
    "target_update_freq": target_update_freq,
    "batch_size": batch_size
}


In [ ]:
import random
import numpy as np
import imageio

def record_video(env, Qnetwork, out_directory, fps=1, max_steps=20):
    """
    Generate a replay video of the agent using a Q-network.
    :param env: Gym environment
    :param Qnetwork: Q-network of our agent
    :param out_directory: Directory to save the video
    :param fps: Frames per second (default is 1 for taxi-v3)
    :param max_steps: Maximum steps for recording
    """
    images = []
    terminated = False
    truncated = False
    state, info = env.reset(seed=random.randint(0, 500))
    state = np.array(state)

    # Render without mode if 'mode' is not supported
    img = env.render()
    images.append(img)
    step_count = 0

    while not (terminated or truncated) and step_count < max_steps:
        # Use Q-network to select the action
        state_input = np.expand_dims(state, axis=0)
        q_values = Qnetwork.predict(state_input)
        action = np.argmax(q_values[0])

        next_state, reward, terminated, truncated, info = env.step(action)
        next_state = np.array(next_state)

        img = env.render()
        images.append(img)
        step_count += 1

        state = next_state

    # Save the video
    imageio.mimsave(out_directory, [np.array(img) for img in images], fps=fps)

# Example usage
video_name = "replay.mp4"
record_video(env, model["Qnetwork_mario"], video_name, fps=1)


In [ ]:
 video_name = "replay.mp4"
 record_video(env, model["Qnetwork_mario"], video_name,1)

1/1 [==============================] - 0s 30ms/step


In [ ]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('replay.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)